# Quiz 3 

In [1]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Question 4

Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

49,58,75,110,112,132,151,276,281,362

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы H0 ⁣:medX=200H_0\colon med X = 200H0​:medX=200 на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет. 

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

*Укажите для критерия знаковых рангов Вилкоксона mode='approx'*

In [2]:
data_lymphoma  = np.array([49,58,75,110,112,132,151,276,281,362])

In [3]:
data_lymphoma 

array([ 49,  58,  75, 110, 112, 132, 151, 276, 281, 362])

In [4]:
print('95%% confidence interval for the mean time: [%f, %f]' % zconfint(data_lymphoma))

95% confidence interval for the mean time: [93.808829, 227.391171]


In [5]:
m0 = 200
stats.wilcoxon(data_lymphoma - m0, alternative="two-sided", mode='approx')

WilcoxonResult(statistic=17.0, pvalue=0.2845026979112075)

## Question 5

В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

22,22,15,13,19,19,18,20,21,13,13,15
и в 9 лесах, где идёт вырубка:

17,18,18,15,12,4,14,15,10

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [6]:
n1 = 12
data_1 = np.array([22,22,15,13,19,19,18,20,21,13,13,15])

n2 = 9
data_2 = np.array([17,18,18,15,12,4,14,15,10])

In [7]:
print('95%% confidence interval for the mean: [%f, %f]' % zconfint(data_1))

95% confidence interval for the mean: [15.503261, 19.496739]


In [8]:
print('95%% confidence interval for the mean: [%f, %f]' % zconfint(data_2))

95% confidence interval for the mean: [10.726721, 16.606613]


In [9]:
stats.mannwhitneyu(data_1, data_2, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

## Question 6

28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

    установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз

    сделайте по 1000 псевдовыборок из каждой выборки.

In [10]:
df = pd.read_csv('challenger.txt', delimiter='\t', index_col=False)
df.rename({'Unnamed: 0':'Time'}, axis='columns', inplace=True)

In [11]:
df.head()

,Time,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time         23 non-null     object 
 1   Temperature  23 non-null     float64
 2   Incident     23 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 680.0+ bytes


In [13]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [14]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [15]:
df_good = df[df['Incident'] == 0]
df_bad = df[df['Incident'] != 0]

In [16]:
df_bad.Temperature

1     21.1
8     13.9
9     17.2
10    21.1
13    11.7
20    23.9
22    14.4
Name: Temperature, dtype: float64

In [17]:
np.random.seed(0)
#np.fromiter(map(odds, get_bootstrap_samples(data_asp, 1000)),dtype=np.float)
samples_good = np.fromiter(map(np.mean, get_bootstrap_samples(df_good['Temperature'].values, 1000)),dtype=np.float)
samples_bad =  np.fromiter(map(np.mean, get_bootstrap_samples(df_bad['Temperature'].values, 1000)),dtype=np.float)

d:\programs\anaconda3\envs\sci\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
d:\programs\anaconda3\envs\sci\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the c

In [18]:
print('95%% confidence interval for times decrease of infarction: %s' %
      str(stat_intervals(samples_good - samples_bad, 0.05)))

95% confidence interval for times decrease of infarction: [1.42299107 7.93861607]


In [19]:
round(1.42299, 4)

1.423

## Question 7

На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

Чтобы получить такое же значение, как мы:

установите random seed = 0; возьмите 10000 перестановок.



In [20]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [21]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [22]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [23]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [25]:
np.random.seed(0)
print('p-value: %.4f', permutation_test(df_bad.Temperature.values, df_good.Temperature.values, max_permutations = 10000))

p-value: %.4f 0.0057
